In [2]:
!pip install pydantic[dotenv]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Getting data from API

## Import

In [3]:
# Libraries
import pandas as pd
import requests
from config import settings

from data import API

In [5]:
currency_from = "USD"
currency_to = "KES"
output_size = "compact"

# make api url
url = (
    "https://www.alphavantage.co/query?"
    "function=FX_DAILY&"
    f"from_symbol={currency_from}&"
    f"to_symbol={currency_to}&"
    f"outputsize={output_size}&"
    f"apikey={settings.api_key}"
    )
print(type(url))

<class 'str'>


In [6]:
# request data from API
response = requests.get(url)

# check response status code
print(f"Response Code: {response.status_code}")
print(response.text[:400])

Response Code: 200
{
    "Meta Data": {
        "1. Information": "Forex Daily Prices (open, high, low, close)",
        "2. From Symbol": "USD",
        "3. To Symbol": "KES",
        "4. Output Size": "Compact",
        "5. Last Refreshed": "2023-06-23 07:55:00",
        "6. Time Zone": "UTC"
    },
    "Time Series FX (Daily)": {
        "2023-06-23": {
            "1. open": "140.33000",
            "2. high": "


In [7]:
# get the data
response_data = response.json()
print(response_data.keys())

# Extract 'Time Series FX (Daily)' from response_data
fx_data = response_data['Time Series FX (Daily)']
print(type(fx_data))

dict_keys(['Meta Data', 'Time Series FX (Daily)'])
<class 'dict'>


In [9]:
# see how the data is organised
fx_data["2023-06-23"]

{'1. open': '140.33000',
 '2. high': '140.33000',
 '3. low': '140.08000',
 '4. close': '140.33000'}

In [10]:
# put data in a dataframe
df = pd.DataFrame.from_dict(fx_data, orient="index", dtype=float)
print(f"df_fx shape: {df.shape}")
print(df.info())
df.head()

df_fx shape: (100, 4)
<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 2023-06-23 to 2023-02-06
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   1. open   100 non-null    float64
 1   2. high   100 non-null    float64
 2   3. low    100 non-null    float64
 3   4. close  100 non-null    float64
dtypes: float64(4)
memory usage: 3.9+ KB
None


,1. open,2. high,3. low,4. close
2023-06-23,140.33,140.33,140.08,140.33
2023-06-22,140.08,140.35,140.08,140.33
2023-06-21,140.13,140.23,139.83,140.20
2023-06-20,140.03,140.18,139.78,140.13
2023-06-19,139.73,140.08,139.73,140.03


In [11]:
# convert df index to DateTime format
df.index = pd.to_datetime(df.index)

# Name index "date"
df.index.name = "date"

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2023-06-23 to 2023-02-06
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   1. open   100 non-null    float64
 1   2. high   100 non-null    float64
 2   3. low    100 non-null    float64
 3   4. close  100 non-null    float64
dtypes: float64(4)
memory usage: 3.9 KB
None


,1. open,2. high,3. low,4. close
date,,,,
2023-06-23,140.33,140.33,140.08,140.33
2023-06-22,140.08,140.35,140.08,140.33
2023-06-21,140.13,140.23,139.83,140.20
2023-06-20,140.03,140.18,139.78,140.13
2023-06-19,139.73,140.08,139.73,140.03


In [12]:
# clean column names
df.columns = [c.split(". ")[1] for c in df.columns]
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2023-06-23 to 2023-02-06
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    100 non-null    float64
 1   high    100 non-null    float64
 2   low     100 non-null    float64
 3   close   100 non-null    float64
dtypes: float64(4)
memory usage: 3.9 KB
None


,open,high,low,close
date,,,,
2023-06-23,140.33,140.33,140.08,140.33
2023-06-22,140.08,140.35,140.08,140.33
2023-06-21,140.13,140.23,139.83,140.20
2023-06-20,140.03,140.18,139.78,140.13
2023-06-19,139.73,140.08,139.73,140.03


## Import Data Using a Function

In [13]:
def get_data(currency_from="USD", currency_to="KES", output_size="full"):
  # create url
  url = (
    "https://www.alphavantage.co/query?"
    "function=FX_DAILY&"
    f"from_symbol={currency_from}&"
    f"to_symbol={currency_to}&"
    f"outputsize={output_size}&"
    f"apikey={settings.api_key}"
    )
  # send request to API
  response = requests.get(url)

  # extract json data from response
  response_data = response.json()

  # put data in a dataframe
   ## data validation step
  if "Time Series FX (Daily)" not in response_data.keys():
    raise Exception(
        f"Invalid API call: check that currency_from sysmbol: '{currency_from}' and currency_to symbol: '{currency_to}' are correct"
    )

  fx_data = response_data['Time Series FX (Daily)']
  df = pd.DataFrame.from_dict(fx_data, orient="index", dtype=float)

  # convert index to datetime fomat: name index as 'date'
  df.index = pd.to_datetime(df.index)
  df.index.name = "date"

  # clean column names
  df.columns = [c.split(". ")[1] for c in df.columns]

  return df


In [14]:
data = get_data("GBP", "USD", "compact")
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2023-06-23 to 2023-02-06
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    100 non-null    float64
 1   high    100 non-null    float64
 2   low     100 non-null    float64
 3   close   100 non-null    float64
dtypes: float64(4)
memory usage: 3.9 KB
None


,open,high,low,close
date,,,,
2023-06-23,1.27477,1.27489,1.26850,1.27125
2023-06-22,1.27677,1.28394,1.27252,1.27433
2023-06-21,1.27631,1.28023,1.26908,1.27661
2023-06-20,1.27949,1.28073,1.27132,1.27631
2023-06-19,1.28255,1.28370,1.27702,1.27941


We are going to use our function in a new python class for automation

### Test API method

In [15]:
# instantiate 'API' instance

api = API()
print("api type:", type(api))

api type: <class 'data.API'>


In [16]:
# get data using your module
fx = api.get_api_data()

print(fx.info())
fx.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2238 entries, 2023-06-23 to 2014-11-24
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    2238 non-null   float64
 1   high    2238 non-null   float64
 2   low     2238 non-null   float64
 3   close   2238 non-null   float64
dtypes: float64(4)
memory usage: 87.4 KB
None


,open,high,low,close
date,,,,
2023-06-23,140.33,140.33,140.08,140.33
2023-06-22,140.08,140.35,140.08,140.33
2023-06-21,140.13,140.23,139.83,140.20
2023-06-20,140.03,140.18,139.78,140.13
2023-06-19,139.73,140.08,139.73,140.03
